# Import the libraries

In [ ]:
import dagsim.base as ds
import numpy as np
import os
from PIL import Image as PILImage
import shutil

# Define the simulation using Python code

## Define the functions

In [ ]:
def create_circular_mask():

    randC1 = np.random.randint(low=10, high=246)
    randC2 = np.random.randint(low=10, high=246)

    Y, X = np.ogrid[:256, :256]
    dist_from_center = np.sqrt((X-randC1)**2 + (Y-randC2)**2)

    mask = (dist_from_center <= 10)*256
    return mask
    

def Ber(U1):
    return np.random.binomial(1, 1-U1)

def BerExp(C, Dnum, Dstr):
    if Dstr=="H":
        out = 0.75*Dnum + 0.5*C + 0.25
    else:
        out = 2.5*Dnum + 1.75*C - 0.25
    out = 1/(1 + np.exp(-out))
    out = np.random.binomial(1, out)
    return out

def drawImage(H, V, R, C, output_path):
    image = np.zeros(shape=(256, 256))
    randInd = np.random.randint(low=1, high=10000)
    if H == 1:
        randPosH = np.random.randint(low=10, high=246)
        image[randPosH - 5:randPosH + 5, :] = 256

    if V == 1:
        randPosV = np.random.randint(low=10, high=246)
        image[:, randPosV - 5:randPosV + 5] = 256

    if C == 1:
        mask = create_circular_mask()
        image = image + mask

    if R == 1:
        TLy = np.random.randint(low=0, high=226)  # y-coordinate of the top-left corner
        TLx = np.random.randint(low=0, high=206)  # x-coordinate of the top-left corner

        image[TLy: TLy+30, TLx: TLx+50] = 256

    image = image + np.random.binomial(1, 0.005, size=(256, 256)) * 256
    image = PILImage.fromarray(image)
    image = image.convert("L")
    image.save(output_path + "/" + str(randInd) + ".png")

## Define and draw the graph

In [ ]:
U1 = ds.Node(name="U1", function=np.random.uniform)
U2 = ds.Node(name="U2", function=np.random.uniform)

H = ds.Node(name="H", function=np.random.binomial, kwargs={"n":1, "p":U1})
C = ds.Node(name="C", function=np.random.binomial, kwargs={"n":1, "p":U2})

V = ds.Node(name="V", function=Ber, kwargs={"U1":U1})

R = ds.Node(name="R", function=BerExp, kwargs={"C":C, "Dnum":H, "Dstr":"H"})
Y = ds.Node(name="Y", function=BerExp, kwargs={"C":C, "Dnum":V, "Dstr":"V"})

Image = ds.Node(name="Image", function=drawImage, kwargs={"H":H, "V":V, "R":R, "C":C})

In [ ]:
graph = ds.Graph(list_nodes=[U1, U2, H, V, C, R, Y, Image])
graph.draw()

## Simulate images

In [ ]:
folder = './images'
if os.path.exists(folder):
    shutil.rmtree(folder)

os.mkdir(folder)

n = graph.simulate(num_samples=50, output_path="./images")

# Define the simulation using YAML

In [ ]:
from dagsim.utils.parser import DagSimSpec

folder = './images'
if os.path.exists(folder):
    shutil.rmtree(folder)

os.mkdir(folder)

data = DagSimSpec("ImagesExample.yaml").parse(verbose=False, draw=False)